In [1]:
from helpers import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_absolute_percentage_error

sns.set_theme()

In [6]:
df = pd.read_csv('prepared_data.csv',parse_dates=['Date'])
df_tea = df.loc[df['Product']=='Tea',['Date','Quantity']].reset_index(drop=True)
del df

split_date = '2023-05-31'

df_train = df_tea[df_tea['Date']<=split_date].copy()

In [7]:
scaler_t = StandardScaler()

df_tea_bc_scaled = df_train.copy()

df_tea_bc_scaled['Quantity_BC_Transform'], lam = boxcox(df_tea_bc_scaled['Quantity'])
df_tea_bc_scaled['Quantity_BC_Scaled'] = scaler_t.fit_transform(df_tea_bc_scaled[['Quantity_BC_Transform']])
df_tea_bc_scaled['Difference'] = df_tea_bc_scaled['Quantity_BC_Scaled'].diff()

In [ ]:
idx = 8

model_df = df_coffee_bc_scaled.copy()
time_lag(model_df,'Difference',steps=idx)
model_df = model_df.dropna()

target = model_df['Difference']

f = []
for i in range(1,8+1):
    f.append(f't-{i}')

features = model_df[f]

features = sm.add_constant(features)

bc_scaled_mlr_model = sm.OLS(target,features).fit()

bc_scaled_mlr_model.summary()

,Date,Quantity,Quantity_BC_Transform,Quantity_BC_Scaled,Difference
0,2023-01-01,286,2.070831,-0.661831,NaN
1,2023-01-02,274,2.067311,-0.833977,-0.172146
2,2023-01-03,296,2.073605,-0.526151,0.307826
3,2023-01-04,266,2.064838,-0.954910,-0.428759
4,2023-01-05,287,2.071115,-0.647957,0.306954
...,...,...,...,...,...
146,2023-05-27,639,2.125833,2.028085,0.426481
147,2023-05-28,459,2.105541,1.035684,-0.992401
148,2023-05-29,435,2.101957,0.860407,-0.175277
149,2023-05-30,473,2.107509,1.131937,0.271530
